In [17]:
import pandas as pd
import numpy as np
import math

# Date: 7-Aug-2018
# Maturity: 21-Jun-2019
# Expiration: 318

# Need to convert expiration in days to expiration in years
period = 318/365

# C = S + P - K*exp(-rT)
r = -(0.0244 + 1)
stock = 379.57
forward = 387.73


In [18]:
spread = pd.read_excel('./ButterflyHomework.xlsx', skiprows=12)
# Need to use skiprows=12 to get the proper dataframe formatting

spread.head()

,strike,Vol 8/7,Fitted Vol,Black-Scholes Call,Bid,Mid,Ask
0,55,0.957070,0.693550,326.325944,325.325944,326.325944,327.325944
1,60,0.902117,0.685908,321.378535,320.378535,321.378535,322.378535
2,65,0.851266,0.678353,316.431076,315.431076,316.431076,317.431076
3,70,0.876616,0.670885,311.858668,310.858668,311.858668,312.858668
4,75,0.758869,0.663504,306.536247,305.536247,306.536247,307.536247


In [19]:
def right_side(p, k):
    return (stock + p - (k*math.exp(r*period)))
for index, row in spread.iterrows():
    print(right_side(row['Mid'], row['strike']))

683.3660689771492
676.3704894812228
669.3748597265586
662.7542812503516
655.3836894333359
648.3956589632287
641.4175829466425
634.8219849192202
627.5389147672145
620.2799622673448
606.902149014112
594.6984211370891
580.9572998801534
566.0910442205525
559.7330616078992
553.9251463873306
546.303650925738
539.4465916009965
532.8629259315655
526.2924086108269
519.5723154748194
512.9013326641095
507.0931615156518
500.3606456991158
493.614281774238
484.9817742948185
480.51110993036093
473.9027164836947
467.9822897941514
460.91180748308057
454.39118092867966
447.78299436811756
441.77492476903655
435.3543385357317
429.108739522081
422.313179265358
416.630056711705
410.4234791418901
404.23732672817823
399.0415599130527
393.2969517035849
386.20192475987
380.1948492230355
373.4496991298685
368.06600421731207
362.76911548338626
356.6098411585532
350.7507934980989
345.2049416823852
339.6598134464442
334.15268559822863
329.4458181743528
323.3134734533976
318.48075908034764
313.0600365850369
306.9639